In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv("../data/freq_data.csv")

# Keep only valid exposure
df = df[(df["Exposure"] > 0) & (df["Exposure"] <= 1)].copy()

# Target
y = df["ClaimNb"].astype(float)

# Predictors (raw)
X_raw = df[[
    "Area",
    "VehPower",
    "VehAge",
    "DrivAge",
    "BonusMalus",
    "VehBrand",
    "VehGas",
    "Density",
    "Region"
]]

# One-hot encode categoricals
X = pd.get_dummies(X_raw, drop_first=True)

# Make sure everything is numeric and no NaN
X = X.astype(float)
X = X.fillna(0.0)

# Offset = log(exposure)
offset = np.log(df["Exposure"].astype(float))

X.head(), y.head(), offset[:5]



(   VehPower  VehAge  DrivAge  BonusMalus  Density  Area_B  Area_C  Area_D  \
 0       5.0     0.0     55.0        50.0   1217.0     0.0     0.0     1.0   
 1       5.0     0.0     55.0        50.0   1217.0     0.0     0.0     1.0   
 2       6.0     2.0     52.0        50.0     54.0     1.0     0.0     0.0   
 3       7.0     0.0     46.0        50.0     76.0     1.0     0.0     0.0   
 4       7.0     0.0     46.0        50.0     76.0     1.0     0.0     0.0   
 
    Area_E  Area_F  ...  Region_R53  Region_R54  Region_R72  Region_R73  \
 0     0.0     0.0  ...         0.0         0.0         0.0         0.0   
 1     0.0     0.0  ...         0.0         0.0         0.0         0.0   
 2     0.0     0.0  ...         0.0         0.0         0.0         0.0   
 3     0.0     0.0  ...         0.0         0.0         1.0         0.0   
 4     0.0     0.0  ...         0.0         0.0         1.0         0.0   
 
    Region_R74  Region_R82  Region_R83  Region_R91  Region_R93  Region_R94  
 

In [5]:
# Add intercept and fit Poisson GLM
X_glm = sm.add_constant(X, has_constant="add")

glm_model = sm.GLM(
    y,
    X_glm,
    family=sm.families.Poisson(),
    offset=offset
)

glm_results = glm_model.fit()
glm_results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                ClaimNb   No. Observations:               676789
Model:                            GLM   Df Residuals:                   676746
Model Family:                 Poisson   Df Model:                           42
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4306e+05
Date:                Sat, 29 Nov 2025   Deviance:                   2.1691e+05
Time:                        19:02:44   Pearson chi2:                 1.79e+06
No. Iterations:                     7   Pseudo R-squ. (CS):            0.01053
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -3.9572      0.047    -85.059      0.000      -4.048      -3.866
VehPower             0.0138      0.003      4.994      0.000       0.008       0.019
VehAge              -0.0388      0.001    -33.184      0.000      -0.041      -0.036
DrivAge              0.0065      0.000     16.565      0.000       0.006       0.007
BonusMalus           0.0224      0.000     72.927      0.000       0.022       0.023
Density          -1.447e-06   3.69e-06     -0.392      0.695   -8.68e-06    5.79e-06
Area_B               0.0520      0.022      2.391      0.017       0.009       0.095
Area_C               0.0860      0.018      4.762      0.000       0.051       0.121
Area_D               0.1844      0.019      9.528      0.000       0.146       0.222
Area_E               0.2255      0.026      8.804      0.000       0.175       0.276
Area_F               0.2309      0.089      2.590      0.010       0.056       0.406
VehBrand_B10        -0.0039      0.037     -0.106      0.916      -0.076       0.068
VehBrand_B11         0.0757      0.040      1.910      0.056      -0.002       0.153
VehBrand_B12         0.1507      0.017      8.615      0.000       0.116       0.185
VehBrand_B13         0.0282      0.041      0.688      0.491      -0.052       0.109
VehBrand_B14        -0.1573      0.079     -1.999      0.046      -0.311      -0.003
VehBrand_B2         -0.0146      0.015     -0.954      0.340      -0.045       0.015
VehBrand_B3         -0.0066      0.022     -0.302      0.763      -0.050       0.036
VehBrand_B4         -0.0373      0.030     -1.252      0.210      -0.096       0.021
VehBrand_B5          0.0664      0.025      2.679      0.007       0.018       0.115
VehBrand_B6         -0.0465      0.028     -1.637      0.102      -0.102       0.009
VehGas_'Regular'     0.0521      0.011      4.754      0.000       0.031       0.074
Region_R21           0.1331      0.082      1.631      0.103      -0.027       0.293
Region_R22           0.0581      0.052      1.122      0.262      -0.043       0.159
Region_R23          -0.0642      0.061     -1.055      0.292      -0.184       0.055
Region_R24           0.0639      0.025      2.608      0.009       0.016       0.112
Region_R25           0.0003      0.045      0.008      0.994      -0.088       0.088
Region_R26          -0.0150      0.049     -0.307      0.759      -0.111       0.081
Region_R31          -0.1087      0.035     -3.082      0.002      -0.178      -0.040
Region_R41          -0.2480      0.046     -5.438      0.000      -0.337      -0.159
Region_R42          -0.0158      0.089     -0.178      0.859      -0.190       0.158
Region_R43          -0.0868      0.134     -0.647      0.517      -0.350       0.176
Region_R52          -0.0136      0.030     -0.449      0.653      -0.073       0.046
Region_R53           0.

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Make sure types are numeric
df["ClaimNb"] = df["ClaimNb"].astype(float)
df["glm_pred"] = df["glm_pred"].astype(float)

# MAE
mae = mean_absolute_error(df["ClaimNb"], df["glm_pred"])

# MSE then RMSE (manual)
mse = mean_squared_error(df["ClaimNb"], df["glm_pred"])
rmse = np.sqrt(mse)

# Poisson deviance (mean per observation)
def poisson_deviance(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    y_pred = np.maximum(y_pred, 1e-9)
    return 2 * np.mean(
        y_true * np.log((y_true + 1e-9) / y_pred) - (y_true - y_pred)
    )

poisson_dev = poisson_deviance(df["ClaimNb"], df["glm_pred"])

mae, rmse, poisson_dev


(0.09888754487613945, 0.2390283060282645, 0.3204939503685755)

In [8]:
df["decile"] = pd.qcut(df["glm_pred"], 10, labels=False)

decile_table = df.groupby("decile", as_index=False).agg(
    actual_mean=("ClaimNb", "mean"),
    predicted_mean=("glm_pred", "mean"),
    count=("ClaimNb", "size")
).sort_values("decile")

decile_table


,decile,actual_mean,predicted_mean,count
0,0,0.022636,0.003630,67679
1,1,0.027999,0.009147,67681
2,2,0.035270,0.017427,67677
3,3,0.042111,0.028806,67679
4,4,0.048139,0.041784,67679
5,5,0.049085,0.054366,67678
6,6,0.054803,0.065796,67679
7,7,0.063890,0.077833,67679
8,8,0.074765,0.092881,67679
9,9,0.113935,0.140962,67679
